<a href="https://colab.research.google.com/github/alvaroferrerrizzo/Employee-Churn-Predictive-Model/blob/master/Reby/alvaro_random/kernel_api_cardpayins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import pandas as pd
import requests
import time
import os
import datetime
from google.cloud import bigquery
import re

In [3]:
cos = ['reby','oiz','avant','tucycle','iberscot','reby_spain','reby_italy']
cos_ids = ['acc_3rjqcmjjct5qey8tstf1','acc_3rjqcudcqm5817xsldx1',
           'acc_3rjqcqpfc144km6bdch1','acc_3rjqcs9fnxk3la3zp5yh',
           'acc_3rjqctd816yhqq7ufj81','acc_3su61k4l3n61dfmsgrah',
           'acc_3su61kkje19lqp9a728h']

In [50]:
from google.colab import auth
auth.authenticate_user()
 
%load_ext google.colab.data_table
project_id = 'reby-cloud'
client = bigquery.Client(project=project_id)

In [4]:
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="reby-cloud.json"

In [39]:
auth_token='live-mob_secretkey_3san8rtxkt5ezlbz34m1_43c759a649130e2512e45d8a071d15c8904b391bd808b5a2'
hed = {'Authorization': 'Bearer ' + auth_token}
final_json = []
url_main = 'https://api.kernelpay.com/v1'

#account_id = 'acc_3rjqcudcqm5817xsldx1'
url_cardpayin_tx = url_main+'/card_payins?limit=100'

print(url_cardpayin_tx)

#url = 'https://api.kernelpay.com/v1/accounts/acc_3rjqcudcqm5817xsldx1/transactions?expand=transfer'
#url_next = 'https://api.kernelpay.com/v1/accounts/acc_3rjqcudcqm5817xsldx1/transactions?expand=transfer'

response = requests.get(url_cardpayin_tx
                        , headers=hed)
#print(response)
#print(response.json())
response_json = response.json()

#response_json['created_at']

https://api.kernelpay.com/v1/card_payins?limit=100


In [40]:
len(response_json)
response_json.keys()

dict_keys(['data', 'has_next', 'cursor_next'])

In [41]:
len(response_json['data'])

100

In [42]:
cursor_next = response_json.get('cursor_next')
has_next = str(response_json.get('has_next'))

In [43]:
print(response_json['has_next'])

True


In [44]:
#working with time
now_ = datetime.datetime.now(tz=None)
print(now_)
days = datetime.timedelta(5)
print(now_-days)

2021-07-12 12:54:26.537605
2021-07-07 12:54:26.537605


In [16]:
created_at = '2021-07-09T12:50:01.819234304Z'
created_at_str = created_at
timestamp_regex = re.findall(
            r"\d{4}-\d{1,2}-\d{1,2}T\d{1,2}:\d{1,2}:\d{1,2}.\d{1,5}",created_at_str)
created_at_regex_unique = timestamp_regex[0]
created_at_formatted = datetime.datetime.strptime(created_at_regex_unique, '%Y-%m-%dT%H:%M:%S.%f')

In [29]:
response_json['data'][0]

{'account_id': 'acc_3su61kkje19lqp9a728h',
 'authentication_redirect_url': 'https://api.reby.co/v2/webhooks/kernelpay/paymentdone/c_3qteetfhxjshx4j54111',
 'authentication_type': 'three_d_secure_2',
 'authentication_url': '',
 'calculated_statement_descriptor': 'EP* Reby',
 'card_id': 'crd_3t92m7p43hat6y3bfjxh',
 'card_info': {'bank': 'ISTITUTO CENTRALE DELLE BANCHE DI CREDITO COOPERATIVO (ICCREA)',
  'brand': 'MasterCard',
  'country': 'IT',
  'expiration_month': 4,
  'expiration_year': 24,
  'fingerprint': 'gj/cEyu+5uB/2IwO',
  'first6': '528287',
  'funding': 'credit',
  'number': '528287******9753'},
 'configure_offline_card': None,
 'created_at': '2021-07-12T12:45:27.872397312Z',
 'decline_code': None,
 'id': 'cpi_3t9hqlyy958482q99nlh',
 'meta': {'_redsys_error_code': '0000',
  'company_id': 'c_3qteetfhxjshx4j54111',
  'top_up_id': 'to_3t9hqlyv8zeeygd235e1',
  'transaction_id': 'tx_3t9hqlyv8vk85qddx8e1',
  'transaction_type': 'pre_ride_charge',
  'user_id': 'usr_3t8my8c5zg2jpjctvb

In [ ]:
j=0
has_next = 'True'
continue_ = 'True'
cardpayins = []

while has_next == 'True' and continue_ == 'True':
#print( "i counter {}".format(i))
    if i == 0:
        fetch_url = url_cardpayin_tx
    elif i>0:
        fetch_url = url_cardpayin_tx+'&cursor='+cursor_next

    response = requests.get(fetch_url, headers=hed)
    response_json = response.json()
    
    for j in range(len(response_json['data'])):

        #reset all values
        api_pinged_at = None
        created_at = None
        updated_at = None
        id = None
        state = None
        authentication_type = None
        account_id = None
        card_id = None
        card_info_bank = None
        card_info_brand = None
        card_info_country = None
        card_info_funding = None
        decline_code = None
        configure_offline_card = None
        meta_redsys_error_code = None
        meta_company_id = None
        meta_top_up_id = None
        meta_transaction_id = None
        meta_transaction_type = None
        meta_user_id = None
        sca_card_id = None
        value_amount = None

        id = response_json['data'][j]['transaction_id']
        sca_card_id = response_json['data'][j]['sca_card_id']
        value_amount = response_json['data'][j]['value']['amount']
        created_at = response_json['data'][j]['created_at']
        updated_at = response_json['data'][j]['updated_at']
        state = response_json['data'][j]['state']
        offline = response_json['data'][j]['offline']
        api_pinged_at = now_
        authentication_type = response_json['data'][j]['authentication_type']
        account_id = response_json['data'][j]['account_id']
        card_id = response_json['data'][j]['card_id']
        card_info_bank = response_json['data'][j]['card_info']['bank']
        card_info_brand = response_json['data'][j]['card_info']['brand']
        card_info_country = response_json['data'][j]['card_info']['country']
        card_info_funding = response_json['data'][j]['card_info']['funding']
        decline_code = response_json['data'][j]['decline_code']
        configure_offline_card = response_json['data'][j]['configure_offline_card']
        meta_company_id = response_json['data'][j]['meta']['company_id']
        meta_top_up_id = response_json['data'][j]['meta']['top_up_id']
        #meta_redsys_error_code = response_json['data'][j]['meta']['_redsys_error_code']
        meta_transaction_id = response_json['data'][j]['meta']['transaction_id']
        meta_user_id = response_json['data'][j]['meta']['user_id']
        meta_transaction_type = response_json['data'][j]['meta']['transaction_type']
    
        cardpayins.append([
                            id,
                           api_pinged_at,
                            created_at,
                            updated_at,
                            state,
                            authentication_type,
                            account_id,
                            card_id,
                            card_info_bank,
                            card_info_brand,
                            card_info_country,
                            card_info_funding,
                            decline_code,
                            configure_offline_card,
                            #meta_redsys_error_code,
                            meta_company_id,
                            meta_top_up_id,
                            meta_transaction_id,
                            meta_transaction_type,
                            meta_user_id,
                            sca_card_id,
                            value_amount])

    created_at_str = created_at
    timestamp_regex = re.findall(
                r"\d{4}-\d{1,2}-\d{1,2}T\d{1,2}:\d{1,2}:\d{1,2}.\d{1,5}",created_at_str)
    created_at_regex_unique = timestamp_regex[0]
    created_at_formatted = datetime.datetime.strptime(created_at_regex_unique, '%Y-%m-%dT%H:%M:%S.%f')



    delta = now_ - created_at_formatted 
    print(f'updated_at_formatted {created_at_formatted}')
    print(f'now_ {now_}')
    print(delta.days)

    continue_ = 'True' if delta.days < 5 else 'False'
    print(continue_)

    cursor_next = response_json.get('cursor_next')
    has_next = str(response_json.get('has_next'))

    print(f'cursor next {cursor_next}')
    
    j = j + 1
    time.sleep(0.1)

In [48]:
cardpayins_df = pd.DataFrame(cardpayins)
if len(cardpayins) >  0:
    cardpayins_df.columns = ['id',
                           'api_pinged_at',
                            'created_at',
                            'updated_at',
                            'state',
                            'authentication_type',
                            'account_id',
                            'card_id',
                            'card_info_bank',
                            'card_info_brand',
                            'card_info_country',
                            'card_info_funding',
                            'decline_code',
                            'configure_offline_card',
                            #meta_redsys_error_code,
                            'meta_company_id',
                            'meta_top_up_id',
                            'meta_transaction_id',
                            'meta_transaction_type',
                            'meta_user_id',
                            'sca_card_id',
                            'value_amount']


In [ ]:
from google.cloud import bigquery

def send_to_bigquery(df,dataset,table):
     client = bigquery.Client(project = 'reby-cloud')
     table_id = 'reby-cloud.'+dataset+'.'+table
     print('table_id: {}'.format(table_id))
     #print(df)
     job_config = bigquery.LoadJobConfig()
     job_config.schema_autodetect = True
     job_config.write_disposition = 'WRITE_TRUNCATE'
     job_config.create_disposition = 'CREATE_IF_NEEDED'
     job = client.load_table_from_dataframe(df, table_id,job_config=job_config)
     # Wait for the load job to complete.
     result = job.result()
     #print(result)
     return 'dataframe appended'

In [51]:
send_to_bigquery(cardpayins_df,'temp_eu','delete_cardpayins')

table_id: reby-cloud.temp_eu.delete_cardpayins


/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'id'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


'dataframe appended'